In [1]:
import os
import json
import re
import scipy.io as spio
import numpy as np
from json_tricks.np import dump, dumps, load, loads
import pandas as pd
import pymworks
import cPickle as pkl
from collections import Counter

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [2]:
source = '/nas/volume1/2photon/projects'
experiment = 'scenes' #'retino_bar' #'gratings_phaseMod'
session = '20171025_CE062' #'20170902_CE054' #'20170825_CE055'
acquisition = 'FOV1' #'FOV1_zoom3x_run2' #'FOV1_planar'
functional_dir = 'functional'
acquisition_dir = os.path.join(source, experiment, session, acquisition)


In [3]:
# Load reference info:
ref_json = 'reference_%s.json' % functional_dir 
with open(os.path.join(acquisition_dir, ref_json), 'r') as fr:
    ref = json.load(fr)

# Load SI meta data:
si_basepath = ref['raw_simeta_path'][0:-4]
simeta_json_path = '%s.json' % si_basepath
with open(simeta_json_path, 'r') as fs:
    simeta = json.load(fs)

file_names = sorted([k for k in simeta.keys() if 'File' in k], key=natural_keys)
nfiles = len(file_names)


In [16]:
# Get PARADIGM INFO:
path_to_functional = os.path.join(acquisition_dir, functional_dir)
paradigm_dir = 'paradigm_files'
path_to_paradigm_files = os.path.join(path_to_functional, paradigm_dir)
path_to_raw = os.path.join(path_to_paradigm_files, 'raw')

# Get SERIAL data:
serialdata_fns = os.listdir(path_to_raw)
serialdata_fns = sorted([f for f in serialdata_fns if 'serial_data' in f], key=natural_keys)
print "Found %02d serial-data files, and %03d TIFFs." % (len(serialdata_fns), nfiles)

# Load MW info:
pydict_jsons = os.listdir(path_to_paradigm_files)
pydict_jsons = sorted([p for p in pydict_jsons if p.endswith('.json') and 'trial_info' in p], key=natural_keys)
print "Found %02d MW files, and %02d ARD files." % (len(pydict_jsons), len(serialdata_fns))

Found 20 serial-data files, and 020 TIFFs.
Found 20 MW files, and 20 ARD files.


In [17]:
serialdata_fns

['20171025_CE062_FOV1_zoom1x_scenes_file002_serial_data_20171025203112624127.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file003_serial_data_20171025203603174189.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file004_serial_data_20171025203848663808.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file005_serial_data_20171025210128614621.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file006_serial_data_20171025210407312220.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file007_serial_data_20171025210650143888.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file009_serial_data_20171025211402581426.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file012_serial_data_20171025212606524437.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file013_serial_data_20171025212845255427.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file014_serial_data_20171025213121624107.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file015_serial_data_20171025213352708094.txt',
 '20171025_CE062_FOV1_zoom1x_scenes_file016_serial_data_20171025213629956272.txt',
 '20

In [18]:
pydict_jsons

['trial_info_20171025_CE062_FOV1_zoom1x_scene_file002.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file003.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file004.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file005.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file006.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file007.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file009.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file012.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file013.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file014.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file015.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file016.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file018.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file019.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file020.json',
 'trial_info_20171025_CE062_FOV1_zoom1x_scene_file021.json',
 'trial_info_20171025_CE

In [126]:
fid = 5
fn = serialdata_fns[fid]
print fn

20171025_CE062_FOV1_zoom1x_scenes_file007_serial_data_20171025210650143888.txt


In [127]:
trialdict_by_file = dict()
#for fid,fn in enumerate(sorted(serialdata_fns, key=natural_keys)):
# fid = 0
# fn = serialdata_fns[fid]
framerate = float(simeta['File001']['SI']['hRoiManager']['scanFrameRate'])

currfile = "File%03d" % int(fid+1)

print "================================="
print "Processing files:"
print "MW: ", pydict_jsons[fid]
print "ARD: ", serialdata_fns[fid]
print "---------------------------------"

### LOAD MW DATA.
with open(os.path.join(path_to_paradigm_files, pydict_jsons[fid]), 'r') as f:
    trials = json.load(f)

### LOAD SERIAL DATA.
ardata = pd.read_csv(os.path.join(path_to_raw, serialdata_fns[fid]), sep='\t')
print ardata.columns

frame_triggers = ardata[' frame_trigger']
arduino_time = ardata[' relative_arduino_time']
bitcodes = ardata[' pixel_clock']

frame_on_idxs = [idx+1 for idx,diff in enumerate(np.diff(frame_triggers)) if diff==1]
#print len(frame_on_idxs)
frame_on_idxs.append(0)
frame_on_idxs = sorted(frame_on_idxs)
print "Found %i frame-triggers:" % len(frame_on_idxs)

Processing files:
MW:  trial_info_20171025_CE062_FOV1_zoom1x_scene_file007.json
ARD:  20171025_CE062_FOV1_zoom1x_scenes_file007_serial_data_20171025210650143888.txt
---------------------------------
Index([u'acquisition_trigger', u' frame_trigger', u' pixel_clock',
       u' abosolute_arduino_time', u' relative_arduino_time',
       u'     absolute_computer_time', u' relative_computer_time'],
      dtype='object')
Found 4650 frame-triggers:


In [128]:
[i for i,b in enumerate(bitcodes) if b==12][0]

15172

In [129]:
### Get bitcodes for each frame:
frame_bitcodes = dict()
for idx,frameidx in enumerate(frame_on_idxs):
    #framenum = 'frame'+str(idx)
    if idx==len(frame_on_idxs)-1:
        bcodes = bitcodes[frameidx:]
    else:
        bcodes = bitcodes[frameidx:frame_on_idxs[idx+1]]
    frame_bitcodes[idx] = bcodes

### Get first frame of trial start:
modes_by_frame = dict()
for frame in frame_bitcodes.keys():
    bitcode_counts = Counter(frame_bitcodes[frame])
    modes_by_frame[frame] = bitcode_counts.most_common(1)[0][0]


In [130]:
first_stim_frame = [k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0][1]
print first_stim_frame


4


In [131]:

### Get all bitcodes and corresonding frame-numbers for each trial:
trialdict = dict()
allframes = sorted(frame_bitcodes.keys()) #, key=natural_keys)
curr_frames = sorted(allframes[first_stim_frame+1:]) #, key=natural_keys)


In [132]:
print first_stim_frame
frame_bitcodes[first_stim_frame]

4


90     15
91     15
92     15
93     15
94     15
95     15
96     15
97     15
98     15
99     15
100    15
101    15
102    15
103    15
104    15
105    15
106    15
107    15
108    15
109    15
110    15
111    15
112    15
Name:  pixel_clock, dtype: int64

In [133]:
trials[trial]['iti_duration']/1E3 # * framerate

4.0

In [134]:

for trial in sorted(trials.keys(), key=natural_keys): #sorted(trials.keys, key=natural_keys):
    #print trial
    trialdict[trial] = dict()
    trialdict[trial]['name'] = trials[trial]['stimuli']['stimulus']
    trialdict[trial]['duration'] = trials[trial]['stim_off_times'] - trials[trial]['stim_on_times']

    if int(trial)>1:
    # Skip a good number of frames from the last "found" index of previous trial.
    # Since ITI is long (relative to framerate), this is safe to do. Avoids possibility that
    # first bitcode of trial N happened to be last stimulus bitcode of trial N-1
        nframes_to_skip = int(((trials[trial]['iti_duration']/1E3) * framerate) - 5)
        #nframes_to_skip = int(((1000/1E3) * framerate) - 5)
        print 'skipping iti...', nframes_to_skip
        curr_frames = allframes[first_frame+nframes_to_skip:]

    first_found_frame = []
    minframes = 4
    for bitcode in trials[trial]['all_bitcodes']:
            looking = True
            while looking is True:
                for frame in sorted(curr_frames):
                    tmp_frames = [i for i in frame_bitcodes[frame] if i==bitcode]
                    consecutives = [i for i in np.diff(tmp_frames) if i==0]

                    if frame>1:
                        tmp_frames_pre = [i for i in frame_bitcodes[int(frame)-1] if i==bitcode]
                        consecutives_pre = [i for i in np.diff(tmp_frames_pre) if i==0]
                    
                    if len(trials[trial]['all_bitcodes'])<3:
                        if len(consecutives)>=minframes:
                            first_frame = frame
                            looking = False
                    else:
                        if frame>1 and len(consecutives_pre)>=minframes:
                            if len(consecutives_pre) > len(consecutives):
                                first_frame = int(frame) - 1
                            elif len(consecutives)>=minframes:
                                first_frame = int(frame)
                            #print "found2...", bitcode, first_frame #len(curr_frames)
                            looking = False

                        elif len(consecutives)>=minframes:
                            first_frame = frame 
                            #print "found...", bitcode, first_frame #len(curr_frames)
                            looking = False

                    if looking is False:
                        break

            first_found_frame.append((bitcode, first_frame)) #first_frame))
            curr_frames = allframes[first_frame+1:] #curr_frames[idx:] #curr_frames[first_frame:]

    #if (first_found_frame[-1][1] - first_found_frame[0][1])/framerate > 2.5:
    #print "Trial %i dur (s):" % int(trial)
    print (first_found_frame[-1][1] - first_found_frame[0][1])/framerate, '[Tial %i]' % int(trial)

    trialdict[trial]['stim_on_idx'] = first_found_frame[0][1]
    trialdict[trial]['stim_off_idx'] = first_found_frame[-1][1]

1.03062987169 [Tial 1]
skipping iti... 173
1.00822487448 [Tial 2]
skipping iti... 173
1.03062987169 [Tial 3]
skipping iti... 173
1.00822487448 [Tial 4]
skipping iti... 173
1.03062987169 [Tial 5]
skipping iti... 173
1.00822487448 [Tial 6]
skipping iti... 173
1.03062987169 [Tial 7]
skipping iti... 173
1.00822487448 [Tial 8]
skipping iti... 173
1.00822487448 [Tial 9]
skipping iti... 173
1.00822487448 [Tial 10]
skipping iti... 173
1.00822487448 [Tial 11]
skipping iti... 173
1.00822487448 [Tial 12]
skipping iti... 173
1.00822487448 [Tial 13]
skipping iti... 173
1.03062987169 [Tial 14]
skipping iti... 173
1.00822487448 [Tial 15]
skipping iti... 173
1.03062987169 [Tial 16]
skipping iti... 173
1.00822487448 [Tial 17]
skipping iti... 173
1.03062987169 [Tial 18]
skipping iti... 173
1.00822487448 [Tial 19]
skipping iti... 173
1.00822487448 [Tial 20]


In [120]:
trials[trial]

{u'all_bitcodes': [14, 1],
 u'end_time_ms': 2616178091.0,
 u'iti_bitcode': 1,
 u'iti_duration': 4000,
 u'start_time_ms': 2616173074.0,
 u'stim_bitcode': 14,
 u'stim_off_times': 15100.0,
 u'stim_on_times': 14084.0,
 u'stimuli': {u'position': [0.0, 0.0], u'scale': 65.0, u'stimulus': u'10'}}

In [121]:
first_found_frame

[(14, 856), (1, 901)]

In [95]:
(first_found_frame[1][1] - first_found_frame[0][1]) / framerate

1.0082248744760032

In [118]:
frame_bitcodes[677]

15175     0
15176    14
15177     3
15178     1
15179     1
15180     1
15181     1
15182     1
15183     1
15184     1
15185     1
15186     1
15187     1
15188     1
15189     1
15190     1
15191     1
15192     1
15193     1
15194     1
15195     1
15196     1
Name:  pixel_clock, dtype: int64

In [36]:
path_to_paradigm_files

'/nas/volume1/2photon/projects/gratings_phaseMod/20170927_CE059/FOV1_zoom3x/functional_subset/paradigm_files'

In [38]:
with open(os.path.join(path_to_paradigm_files, 'parsed_trials.pkl'), 'rb') as f:
    trialdict = pkl.load(f)

In [94]:
ref['frame_idxs'].index(2403) + 1

1443

In [50]:
ref['frame_idxs'][0:5]

[1, 2, 3, 6, 7]